# Data class example

In [ ]:
import numpy as np
from skimage import io
import json
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models

from torchvision.datasets.utils import download_file_from_google_drive
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader,Dataset
from PIL import Image
import os
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
from os import listdir
from os.path import isfile, join
output_feat = 2048

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Utility functions

In [ ]:
def get_filepaths(directory, format):
    """
    This function will generate the file names in a directory 
    tree by walking the tree either top-down or bottom-up. For each 
    directory in the tree rooted at directory top (including top itself), 
    it yields a 3-tuple (dirpath, dirnames, filenames).
    """
    file_paths = []  # List which will store all of the full filepaths.

    # Walk the tree.
    for root, directories, files in os.walk(directory):
        for filename in files:
            if format in filename: 
            # Join the two strings in order to form the full filepath.
                filepath = os.path.join(root, filename)
                file_paths.append(filepath)  # Add it to the list.
            else:
                pass
    return file_paths  # Self-explanatory.

### Dataloader demo

In [34]:
train_transform = transforms.Compose([
      transforms.ToTensor()               
])

valid_transform = transforms.Compose([
    transforms.ToTensor(),           
])

class DengueSet_loader(Dataset):
    def __init__(self, root, transform = None):
        self.root = root
        self.imgs = list(sorted(get_filepaths(root, ".tiff")))
        self.targets = list(sorted(get_filepaths(root, ".json")))
    def __len__(self):
        return len(self.imgs)
    def __getitem__(self, ind):
        image_path = self.imgs[ind]
        target_path = self.targets[ind]
        # Read data
        image = io.imread(image_path)
        metadata = json.load(open(target_path))
        
        #if self.transform:
        #  image = self.transform(image)
        
        return image, metadata

  
root = "./DATASET/"
train_set = DengueSet_loader(root, train_transform)
loader = DataLoader(train_set, batch_size=1)
print(len(loader))
cont = 0
for x, y in loader:
        print(x.shape,y)
        cont+=1
        if cont==1:
                break
        

1560
torch.Size([1, 744, 748, 12]) {'image_path': ['DATASET/images/23001/image_2016-01-03.tiff'], 'municipality_code': tensor([23001]), 'date': tensor([201601]), 'labels': {'cases': tensor([31]), 'binary_classification': tensor([1]), 'incidence_ratio': tensor([0]), 'multilabel': tensor([0])}, 'metadata': {'environmental_data': {'temperature': [tensor([30.7032], dtype=torch.float64)], 'precipitation': [tensor([0.1678], dtype=torch.float64)], 'elevation': tensor([36.], dtype=torch.float64)}, 'socioeconomic_data': {'Population': tensor([471724]), 'Age0-4(%)': tensor([7.7600], dtype=torch.float64), 'Age5-14(%)': tensor([26.3100], dtype=torch.float64), 'Age>30(%)': tensor([48.6900], dtype=torch.float64), 'AfrocolombianPopulation(%)': tensor([1.7000], dtype=torch.float64), 'IndianPopulation(%)': tensor([0.7100], dtype=torch.float64), 'PeoplewithDisabilities(%)': tensor([2.6900], dtype=torch.float64), 'Peoplewhocannotreadorwrite(%)': tensor([5.9300], dtype=torch.float64), 'Secondary/HigherEdu

In [36]:
target_path = "/home/sebasmos/Desktop/DengueSet/DATASET/annotations/23001/image_2018-12-02.json"
metadata = json.load(open(target_path))
metadata

{'image_path': 'DATASET/images/23001/image_2018-12-02.tiff',
 'municipality_code': 23001,
 'date': 201849,
 'labels': {'cases': 77,
  'binary_classification': 1,
  'incidence_ratio': 0,
  'multilabel': 1},
 'metadata': {'environmental_data': {'temperature': [29.652235658413748],
   'precipitation': [0.0],
   'elevation': 36.0},
  'socioeconomic_data': {'Population': 490935,
   'Age0-4(%)': 7.76,
   'Age5-14(%)': 26.31,
   'Age>30(%)': 48.69,
   'AfrocolombianPopulation(%)': 1.7,
   'IndianPopulation(%)': 0.71,
   'PeoplewithDisabilities(%)': 2.69,
   'Peoplewhocannotreadorwrite(%)': 5.93,
   'Secondary/HigherEducation(%)': 59.93,
   'Employedpopulation(%)': 36.46,
   'Unemployedpopulation(%)': 4.61,
   'Peopledoinghousework(%)': 17.02,
   'Men(%)': 48.51,
   'Women(%)': 51.49,
   'Householdswithoutwateraccess(%)': 10.07,
   'Householdswithoutinternetaccess(%)': 52.84,
   'Buildingstratification1(%)': 60.2975,
   'Buildingstratification2(%)': 14.4266,
   'Buildingstratification3(%)': 6.